# 1. 추천 시스템
- 콘텐츠 기반 필터링(Content based filtering)
- 협업 기반 필터링(Collaborative Filtering): 최급접 이웃필터링, 잠재요인 필터링

# 2. 콘텐츠 기반 필터링(Content based filtering)
- 사용자가 특정한 아이템을 매우 선호하는 겨웅, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
- 사용자 선호 프로파일
- 사용자가 선호하는 영화의 장르, 배우, 감독에 맞게 유사한 영화를 추천

# 3. 최근접 이웃 협업 필터링(메모리 협업)
- 취향이 비슷한 사람들을 기반으로 추천(취향이 비슷한 친구에게 물어보기)
- 사용자가 매긴 평점 정보, 상품 구매이력과 같이 사용자 행동양식을 기반으로 추천
- 목표: 사용자-아이템 평점 메트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측평가 하는 것
- 아직 안본 영화를 예측해서 취향에 맞게 추천해주기
- 최근접이웃, 잠재요인 두 방식 모두 사용자-아이템 평점 행렬 데이터에만 의지해 추천
- 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 높다.

## 메모리 협업필터링

### 사용자 기반(User-User)
- 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
- TOP-N: 사용자가 좋아하는 아이템을 추천
- 특정 사용자와 타 사용자간의 유사도(similarity)를 측정한 뒤 가장 유사도가 높은 TOP-N 사용자를 추출해 그들이 선호하는 아이템을 추천

### 아이템 기반(Item-Item)
- 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다.
- 사용자들이 아이템을 좋아하는지/싫어하는지
- 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 높다.

# 4. 잠재요인 협업 필터링
- 사용자-아이템 평점행렬 데이터 속에 숨어있는 '잠재요인'을 추출해 추천 예측

- 대규모의 다차원행렬 SVD 같은 차원감소 기법으로 분해하는 과정에서 잠재요인을 추출
- P.592-594\
원본-     사용자 - 아이템 평점 행렬: R\
매트릭스- 사용자 - 잠재 요인 행렬: P\
매트릭스- 잠재요인 - 아이템 행렬: Q.T\
내적 결과값 예측 => R = (P * QT)

### 다차원 희소행렬
- 1. 사용자-아이템 행렬을 저차원 밀집 행렬의 사용자-잠재요인 행렬
- 2. 아이템-잠재요인 행렬의 전치(잠재요인-아이템 행렬)로 분해
- 잠재요인 협력 필터링 알고리즘
- : 분해한 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성
- 사용자- 잠재 요인 행렬과 아이템-잠재요인 행렬의 전치행렬(잠재요인-아이템행렬)로 분해한 데이터 세트를 다시 내적의 곱으로 결합하면서 사용자가 예측하지 않은 아이템에 대한 평점을 도출

### 행렬분해(Matrix Factorization)
- 다차원 매트릭스를 저차원 매트릭스로 분해(SVD, NMF)
- 사용자-K차원 잠재요인행렬P와 / K차원 잠재 요인-아이템행렬(Q.T)으로 분해
- R은 N*M 차원으로 구성
- P는 M*k
- Q.T는 K*N
- Q는 아이템-잠재요인 행렬
- Q.T는 Q의 전치 행렬인 잠재요인-아이템 행렬
- R = P * Q.T

### 경사하강법(SGD)을 이용한 행렬분해 

In [15]:
# p.599
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q초기화, 잠재 요인 차원 K는 3으로 설정
R= np.array([[4, np.NaN, np.NaN, 2, np.NaN],
            [np.NaN, 5, np.NaN, 3, 1],
            [np.NaN, np.NaN, 3, 4, 4],
            [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

# P와 Q행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력한다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q= np.random.normal(scale=1./K, size=(num_items, K))

In [35]:
# get_rmse()함수는 실제 R행렬의 널이 아닌 헹렬 값의 위치 인덱스를 추출해 
# 이 인덱스에 있는 실제 R 행렬값과, 
# 분해된 P,Q를 이용해 다시 조합된 예측 행렬값의 RMSE값을 반환
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬과 P, Q.t의 내적으로 예측 R행렬 생성
    # np.dot은 행렬의 곱
    full_pred_matrix = np.dot(P, Q.T)
    
   # 실제 R행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [39]:
# SGD기반의 행렬분해 수행
# R>0인 행 위치, 열위치, 값을 non_zeros 리스트에 저장
non_zeros= [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] >0]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SDG기법으로 P와 Q매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측값의 차이인 오류 값 구함
        eij= r-np.dot(P[i,:], Q[j,:] -r_lambda * P[i,:])
        # Regularization을 반영한 SGD업데이트 공식 적용
        P[i,:]= P[i,:] + learning_rate * (eij * Q[j, :] - r_lambda * P[i,:])
        Q[j,:]= Q[j,:] + learning_rate * (eij * P[i, :] - r_lambda * Q[i,:])
        
        rmse= get_rmse(R, P, Q, non_zeros)
        if (step % 50)==0:
            print("### iteration step:", step, "rmse: ", rmse)

### iteration step: 0 rmse:  0.03234370977609321
### iteration step: 0 rmse:  0.03219884931383537
### iteration step: 0 rmse:  0.03230337616581673
### iteration step: 0 rmse:  0.03242090285045925
### iteration step: 0 rmse:  0.032356736001100735
### iteration step: 0 rmse:  0.03245721540968807
### iteration step: 0 rmse:  0.032583139226079465
### iteration step: 0 rmse:  0.03253308008561175
### iteration step: 0 rmse:  0.032688299416348585
### iteration step: 0 rmse:  0.03255862108226665
### iteration step: 0 rmse:  0.032411791203371634
### iteration step: 0 rmse:  0.03233996941538673
### iteration step: 50 rmse:  0.032259595784146965
### iteration step: 50 rmse:  0.03211545695412944
### iteration step: 50 rmse:  0.03221981314584846
### iteration step: 50 rmse:  0.032336655708839535
### iteration step: 50 rmse:  0.032273145439031654
### iteration step: 50 rmse:  0.03237387010452517
### iteration step: 50 rmse:  0.03249875221883673
### iteration step: 50 rmse:  0.03244806720217455
### i

In [40]:
# 분해된 P와 Q함수를 P*Q.T로 예측행렬 만들어 출력
pred_matrix= np.dot(P, Q.T)
np.round(pred_matrix,3)

array([[4.01 , 1.53 , 1.156, 2.014, 1.606],
       [5.674, 5.032, 0.882, 3.039, 1.046],
       [4.644, 0.471, 3.031, 4.036, 4.045],
       [5.005, 2.029, 1.032, 2.019, 1.497]])

# 5. 콘텐츠 기반의 필터링 실습 - TMDB5000영화데이터세트

## 장르 속성을 이용한 영화 콘텐츠 기반 필터링

In [122]:
# p.603
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies= pd.read_csv("C:/Users/user/머신러닝/data/tmdb_5000_movies.csv")
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [123]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [124]:
# 주요칼럼을 추출해 데이터프레임으로 변환
# 방법1
movies_df=movies.iloc[:, [3,17,1,18,19,8,4,7]]
movies_df[:2]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."


In [125]:
# 방법2
movies_df=movies[['id', 'title','genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [126]:
# df으로 만들어 질 떄는 단순히 문자열 형태로 로딩되므로 
#이 칼럼은 가공하지 않고는 필요한 정보를 추출할 수가 없다.

pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [127]:
# 개별 장르 명칭은 딕셔너리 key인 'name'으로 추출
pd.set_option('max_colwidth',100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [128]:
# 문자열을 객체로 변환(리스트)
from ast import literal_eval
movies_df['genres']= movies_df['genres'].apply(literal_eval)
movies_df['keywords']= movies_df['keywords'].apply(literal_eval)
# 이제 genres 칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다.

In [129]:
# genres칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르로 구성된 객체를 가짐
movies_df['genres']=movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정 CountVectorizer변환
- genres칼럼은 여러개의 개별 장르가 리스트로 구성
- 장르가 list로 되어있다면 어떻게 장르별 유사도를 측정할까?
- genres를 문자열로 변경 한 뒤 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교

In [131]:
# 문자열로 변환
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열 변환
movies_df['genres_literal']=movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect= CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape


# CountVectorizer로 변환해 4803개의 레코드, 276개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어짐

(4803, 276)

### 코사인유사도 계산 cosine_similarity

In [133]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


### genre_sim

In [173]:
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
genre_sim_sorted_ind[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

In [174]:
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

### 장르 콘텐츠 필터링을 이용한 영화 추천

In [143]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes] 

In [144]:
# 상위 10개 장르별 추천 영화
similar_movies= find_sim_movie(movies_df, genre_sim_sorted, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


### vote_average
- 주의: 0-10점 만점의 점수로 돼 있음
- 여러관객이 평가한 평점을 평균한것임
- 왜곡이 있을 수 있음

In [147]:
# 오름차순으로 movies_df 10개를 를 정렬해 왜곡된 데이터 확인
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


### 가중치가 부여된 평점
- v/(v+m)*0.6  m/(v+m)*0.4

In [153]:
# p.610
C= movies_df['vote_average'].mean()
m= movies_df['vote_count'].quantile(0.6)
print(C)
print(m)

6.092171559442011
370.1999999999998


In [161]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 
movies_df['weighted_vote']

0       7.166301
1       6.838594
2       6.284091
3       7.541095
4       6.098838
          ...   
4798    6.290894
4799    6.089611
4800    6.106650
4801    6.084894
4802    6.100782
Name: weighted_vote, Length: 4803, dtype: float64

In [162]:
movies_df[['title', 'vote_average','weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


### find_sim_movies()

In [175]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


## 8. Surpeise

In [1]:
pip install surprise --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 모듈 임포트
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
# 데이터 내려받기
data=Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\user/.surprise_data/ml-100k


In [5]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [6]:
algo= SVD(random_state=0)
algo.fit(trainset)

In [9]:
# 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행
# test()메서드 수행 
# 사용자-아이템 평점 데이터 세트 전체에 대해 추천을 예측
predictions= algo.test(testset)
print(type(predictions), len(predictions))
predictions[:5]

<class 'list'> 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [10]:
# prediction 객체의 uid, iid, est속성을 추출
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [11]:
# predict()메서드 수행 
# 개별 사용자의 아이템 추천 평점을 예측
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야함
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

# prediction 객체는 사용자 아이디(uid), 아이템 아이디(iid), 실제 평점(r_ui), 예측 평점(est)를 튜플 형태로 가진다.
# prediction 객체의 details속성은 추천 예측을 할 수 없는 경우 True, 아닌 경우 False이다.

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [12]:
# 추천예측평점과 실제예측평점의 차이 평가
# accuracy모듈은 RMSE, MSE등 방법으로 추천 시스템의 성능 평가 정보를 제공
# RMSE평가 결과 확인
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

In [15]:
# OS파일 데이터를 Surprise데이터 세트로 로딩
import pandas as pd

ratings= pd.read_csv('C:/Users/user/머신러닝/data/ml-latest-small/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('C:/Users/user/머신러닝/data/ratings_noh.csv', index=False, header=False)

In [22]:
# noh파일에서 앞의 3개의 칼럼만 로딩되고 timestamp칼럼은 제외된다
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('C:/Users/user/머신러닝/data/ratings_noh.csv', reader=reader)

In [ ]:
#Reader클래스의 주요 파라미터
# line_format(string): 칼럼의 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
# sep(char): 칼럼을 분리하는 분리자. 디폴트는 '\t'이고, 판다스DataFrame에서 입력받을 경우에는 기재할 필요가 없다.
# rating_scale(tuple, optional): 평점값의 최소~최대 평점을 설정. 디폴트는(1,5)이지만 ratings.csv파일의 경우 최소 평점이 0.5, 최대 평점이 5이므로(0.5, 5)로 설정

### SVD행렬분해

In [24]:
# 예측평점을 구하고 실제 평점 데이터를 RMSE로 평가
trainset, testset= train_test_split(data, test_size=.25, random_state=0)
# 수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo= SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점예측 후 RMSE 평가
algo.fit(trainset)
predictions= algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 판다스 DataFrame에서 Suprise 데이터 세트로 로딩

### SVD 추천예측코드작성

In [28]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('C:/Users/user/머신러닝/data/ml-latest-small/ml-latest-small/ratings.csv')
reader= Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data= Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo= SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Suprise 추천 알고리즘 클래스
- SVD: 행렬분해를 통한 잠재 요인 협업 필터링을 위한 SVD알고리즘
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN알고리즘
- BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD베이스라인 알고리즘

In [29]:
# Suprise SVD의 비용함수는 사용자 베이스라인(Baseline) 편향성을 감안한 평점 예측에 Regular-ization을 적용한 것이다.

### SVD클래스의 입력 파라미터
- n_factors: 잠재요인 K의 개수. 디폴트는 100, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
- n_epochs: SGD(Stochastic Gradient Descent) 수행 시 반복 횟수, 디폴트는 20
- biased(bool): 베이스라인 사용자 편행 적용 여부이며, 디폴트는 True

### 베이스라인 평점
-  각 개인이 평점을 부여하는 성향을 반영하여 평점을 계산하는 방식
-  전체평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
- 전체 평균평점= 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자 편향 점수= 사용자별 아이템 - 전체 평균한 값
- 아이템 편향 점수= 아이템별 평점 평균 값 -전체 평균 평점

## 교차검증과 하이퍼파라미터 튜닝

### cross_validation()
- 함수는 srprise.model_selection 모듈 내에 존재하며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차검증을 수행

In [31]:
#df를 이용해 데이터 학습/검증 폴드 데이터 세트로 분리, 교차검증을 수행,
# RMSE, MAE로 성능평가

from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings= pd.read_csv('C:/Users/user/머신러닝/data/ml-latest-small/ml-latest-small/ratings.csv') #rating data in pandas df
reader= Reader(rating_scale=(0.5,5.0))
data= Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo= SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8758  0.8696  0.8755  0.8806  0.8708  0.8745  0.0039  
MAE (testset)     0.6723  0.6679  0.6696  0.6775  0.6706  0.6716  0.0033  
Fit time          3.12    3.15    3.15    3.13    3.16    3.14    0.01    
Test time         0.09    0.29    0.08    0.08    0.08    0.13    0.08    


{'test_rmse': array([0.8757737 , 0.86959462, 0.8754576 , 0.88062916, 0.87082095]),
 'test_mae': array([0.67234547, 0.66785633, 0.66960976, 0.67748476, 0.67057724]),
 'fit_time': (3.1226577758789062,
  3.150580883026123,
  3.149553060531616,
  3.1326282024383545,
  3.1605536937713623),
 'test_time': (0.08574247360229492,
  0.29421329498291016,
  0.08477330207824707,
  0.08377623558044434,
  0.08477354049682617)}

### GridSearchCV

In [32]:
from surprise.model_selection import GridSearchCV
# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid= {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# cv를 3개의 폴드 세트로 지정, 성능 평가는 rmse,mse로 수행하도록 gridsearchcv구성
gs= GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'],cv=3)
gs.fit(data)

#최고 RMSE Evaluation 점수롸 그떄의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.876291928750689
{'n_epochs': 20, 'n_factors': 50}


## Surprise를 이용한 개인화 영화 추천 시스템 구축
- Surprise 패키지로 학습괸 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 (관람하지 않은) 영화 중에서 개인 취향에 가장 적절한 영화를 추천

In [33]:
# Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수 없다
# 따라서 데이터 세트를 그대로 fit()에 적용한 다음 코드는 오류를 일으킨다

# 오류 발생 코드
data= Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo= SVD(random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [35]:
# 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds클래스를 이용하면 됨
# DatasetAutoFolds 객체를 생성한 뒤 bulid_full_trainset()메서드를 호출하면 전체 데이터를 학습 데이터 세트로 만들 수 있음

from surprise.dataset import DatasetAutoFolds

reader= Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv파일 기반으로 생성
data_folds= DatasetAutoFolds(ratings_file='C:/Users/user/머신러닝/data/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset= data_folds.build_full_trainset()

In [37]:
# DDatasetAutoFolds의 build_full_trainset() 메서드를 이용해 생성된 학습데이터 기반으로 학습을 수행
algo= SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [39]:
# user9가 아직 평점을 매기지 않은 영화를 movied 42로 선정한 뒤 예측 평점을 계산
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies= pd.read_csv('C:/Users/user/머신러닝/data/ml-latest-small/ml-latest-small/movies.csv')

#userId=9 의 movieId의 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds= ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [40]:
# userId9 사용자의 추천 예상 평점은 predict() 메서드를 이용하면 알 수 있음
uid= str(9)
iid= str(42)
pred= algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [54]:
# 추천 대상이 되는 영화를 추출
def get_unseen_surprise(ratings, movies, userId):
    # 입력 값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies= ratings[ratings['userId']==userId]['movieId'].tolist()
    
    # 모든 영화의 movieId를 리스트로 생성
    total_movies= movies['movieId'].tolist()
    
    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
          '전체 영화 수:', len(total_movies))
    return unseen_movies
unseen_movies= get_unseen_surprise(ratings, movies,9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [61]:
# 학습된 알고리즘인 SVD를 이용해 높은 예측 평점을 가진 순으로 영화 추천
# TOP-N개의 prediction 객체에서 영화 아이디, 영화 제목, 예측평점 정보를 추출해 반환

def recommed_movie_by_surprise(algo, userId, unseed_movies, top_n=10):
    # 알고리즘 객체의 predict()메서드를 평점이 없는 영화에 반복 수행 한 결과를 list객체로 저장
    predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseed_movies]
    
    #predictions list객체는 surprise의 predictions 객체를 원소로 쓰고 있음
    #[prediction(udi='9', iid='1', est=2.69), predprediction(udi='9', iid='2', est=2.98),,,]
    
    # 이를 est값으로 정렬하기 위해서 아래의 sortkey_est함수를 정의함
    #sortkey_est함수는 list객체의 sort()함수의 키 값으로 사용되어 정렬수행
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est()반환값의 내림차순으로 정렬 수행하고 top_n개의 최상의 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    #top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids= [int(pred.iid) for pred in top_predictions]
    top_movie_rating= [pred.est for pred in top_predictions]
    top_movie_titles= movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds=[(id, title, rating) for id, title, rating in\
                    zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies= get_unseen_surprise(ratings, movies,9)
top_movie_preds= recommed_movie_by_surprise(algo,9, unseen_movies, top_n=10)

print('### TOP-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
    print(top_movie[1], ";", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### TOP-10 추천 영화 리스트 ###
Usual Suspects, The (1995) ; 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) ; 4.281663842987387
Pulp Fiction (1994) ; 4.278152632122759
Silence of the Lambs, The (1991) ; 4.226073566460876
Godfather, The (1972) ; 4.1918097904381995
Streetcar Named Desire, A (1951) ; 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) ; 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) ; 4.108009609093436
Goodfellas (1990) ; 4.083464936588478
Glory (1989) ; 4.07887165526957
